Import the pre trained model

In [1]:
import tensorflow as tf
import tensorflow.keras

In [2]:
model = tf.keras.models.load_model('trained_models/CNN_LSTM1.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 128, 102, 64)      256       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 128, 51, 64)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 128, 49, 64)       12352     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 128, 12, 64)       0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 128, 10, 128)      24704     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 128, 2, 128)       0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 128, 256)          0

Import the pre processed data

In [3]:
import numpy as np
x_train = np.load('processed_data/first_experiment/x_train.npy')
x_test = np.load('processed_data/first_experiment/x_test.npy')
y_train = np.load('processed_data/first_experiment/y_train.npy')
y_test = np.load('processed_data/first_experiment/y_test.npy')

In [4]:
y_test.shape

(113, 7)

In [5]:
_, accuracy_test_original = model.evaluate(x_test, y_test)
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))

4/4 [==============================] - 3s 197ms/step - loss: 0.9742 - accuracy: 0.6726
Accuracy on original test data: 67.26%


Make ART model

In [7]:
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod, CarliniL0Method

In [12]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
classifier = TensorFlowV2Classifier(model=model, nb_classes=7, input_shape=(128, 104, 1), loss_object=loss_object)

In [13]:
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.3)

In [14]:
x_test_adv = attack_fgsm.generate(x_test)

In [15]:
_, accuracy_test_attack = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))

print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test_attack * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))

4/4 [==============================] - 0s 92ms/step - loss: 1.0173 - accuracy: 0.6018
Accuracy on adversarial test data: 60.18%
Average perturbation: 0.22


In [ ]:
attack_cw = CarliniL0Method(classifier=classifier)

x_test_adv = attack_cw.generate(x_test)

In [ ]:
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - x_test)))
print('Accuracy on original test data: {:4.2f}%'.format(accuracy_test_original * 100))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}'.format(perturbation))